# Biostat 257 Homework 2

### Yixuan Zhou (UID 505 524 487)

**Due Apr 29 @ 11:59PM**

In [ ]:
versioninfo()

In [1]:
# load libraries
using BenchmarkTools, DelimitedFiles, Images, LinearAlgebra, Random

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1423
┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1423


## Q1. Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Lee and Seung suggest an iterative algorithm with multiplicative updates
$$
v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t+1)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t+1)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}
$$
that will drive the objective $L^{(t)} = L(\mathbf{V}^{(t)}, \mathbf{W}^{(t)})$ downhill. Superscript $t$ indicates iteration number. In following questions, efficiency (both speed and memory) will be the most important criterion when grading this problem.

### Q1.1 Develop code

Implement the algorithm with arguments: $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.

In [ ]:
function nnmf(
    X       :: Matrix{T}, #T: (specifies type of element in the matrix) abstract float
    r       :: Integer; # any type of integer
    # X and r are required arguments. After the semicolon are optional arguments
    maxiter :: Integer = 1000, 
    tolfun  :: Number = 1e-4,
    V       :: Matrix{T} = rand(T, size(X, 1), r), 
    W       :: Matrix{T} = rand(T, r, size(X, 2))
    # V and W are starting values 
    ) where T <: AbstractFloat
    # implementation
    # evaluate obj at starting point
    # initialization
    for iter in 1:maxiter
        # TODO update V 
        # TODO update W
        # these two steps are of interest
        # evaluate obj
        objold = obj # store old obj
        obj = ... 
        niter = maxiter
        # check convergence (criterion in 1.3)
        if abs(obj - objold) <= tolfun * (abs(objold) + 1)
            niter = iter
            break
        end     
    end
    # Output: returns whatever is on the last line
    V, W, obj, niter
end

In [20]:
#X = readdlm("nnmf-2429-by-361-face.txt")
X = abs.(randn(4,6))
r = 3
maxiter = 2
tolfun = 1e-4
V = rand(size(X, 1), r)
W = rand(r, size(X, 2))

3×6 Matrix{Float64}:
 0.750322  0.599032  0.281214  0.74488   0.896104  0.424931
 0.377483  0.1875    0.765121  0.54295   0.986991  0.248434
 0.668728  0.789416  0.88107   0.576785  0.896508  0.768132

In [21]:
V

4×3 Matrix{Float64}:
 0.41563   0.942924   0.228512
 0.599111  0.790634   0.844326
 0.6375    0.0649066  0.473245
 0.726727  0.468496   0.724374

In [22]:
W

3×6 Matrix{Float64}:
 0.750322  0.599032  0.281214  0.74488   0.896104  0.424931
 0.377483  0.1875    0.765121  0.54295   0.986991  0.248434
 0.668728  0.789416  0.88107   0.576785  0.896508  0.768132

In [ ]:
# TODO update V 

# TODO update W
# these two steps are of interest
# evaluate obj
objold = obj # store old obj
obj = ... 
niter = maxiter
# check convergence (criterion in 1.3)
if abs(obj - objold) <= tolful * (abs(objold) + 1)
    niter = iter
    break
end     


In [19]:
old = zeros(4,6)
@benchmark old = norm(X - V*W)

BenchmarkTools.Trial: 10000 samples with 199 evaluations.
 Range (min … max):  418.754 ns …  17.553 μs  ┊ GC (min … max): 0.00% … 95.24%
 Time  (median):     443.090 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   497.939 ns ± 615.954 ns  ┊ GC (mean ± σ):  5.19% ±  4.19%

  ▂▇▇█▄▅▅▅▂ ▁▂▁▂▂▁▁         ▁                                   ▂
  █████████████████████▇▇██▇███▆▇▇▆▇▆█▆▇▇▆▆▆▆▅▆▅▅▅▆▄▅▆▄▄▅▃▅▄▄▃▄ █
  419 ns        Histogram: log(frequency) by time        874 ns <

 Memory estimate: 528 bytes, allocs estimate: 3.

### Q1.2 Data

Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  

Read in the [`nnmf-2429-by-361-face.txt`](https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm`](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/#Delimited-Files) function, and display a couple sample images, e.g., using the [Images.jl](https://juliaimages.org/stable/) package.

In [ ]:
X = readdlm("nnmf-2429-by-361-face.txt")
#V0full = readdlm("")
#W0full = raddlm("")
colorview(Gray, reshape(X[1, :], 19, 19))

In [ ]:
colorview(Gray, reshape(X[5, :], 19, 19))

### Q1.3 Correctness and efficiency

Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw2/W0.txt)) and stopping criterion
$$
\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

**Hint**: When I run the following code using my own implementation of `nnmf`
```julia
for r in [10, 20, 30, 40, 50]
    println("r=$r")
    V0 = V0full[:, 1:r]
    W0 = W0full[1:r, :]
    @time V, W, obj, niter = nnmf(X, r; V = V0, W = W0)
    println("obj=$obj, niter=$niter")
end
```
the output is
```
r=10
  1.047598 seconds (20 allocations: 6.904 MiB)
obj=11730.38800985483, niter=239
r=20
  1.913147 seconds (20 allocations: 7.120 MiB)
obj=8497.222317850326, niter=394
r=30
  2.434662 seconds (20 allocations: 7.336 MiB)
obj=6621.627345486279, niter=482
r=40
  3.424469 seconds (22 allocations: 7.554 MiB)
obj=5256.663870563529, niter=581
r=50
  4.480342 seconds (23 allocations: 7.774 MiB)
obj=4430.201581697291, niter=698
```
Since my laptop is about 6-7 years old, I expect to see your run time shorter than mine. Your memory allocation should be less or equal to mine.

#### Class Solution

**Step 1 Protyping**

In [5]:
function nnmf_prototype(
    X       :: Matrix{T}, #T: (specifies type of element in the matrix) abstract float
    r       :: Integer; # any type of integer
    maxiter :: Integer = 1000, 
    tolfun  :: Number = 1e-4,
    V       :: Matrix{T} = rand(T, size(X, 1), r), 
    W       :: Matrix{T} = rand(T, r, size(X, 2)),
    verbose :: Bool = false
    ) where T <: AbstractFloat
    # implementation
    obj = norm(X - V * W)^2
    
    if verbose
        println("iter = 0, obj = $obj")
    end
    
    niter = maxiter
    for iter in 1:maxiter
        # TODO update V 
        # optimize 
        V .= V .* (X * W') ./ (V * W * W')
        # the dot in equal stores result on the right to the left (i.e. V is updated)
        # TODO update W
        # optimize
        W .= W .* (V' * X) ./ (V' * V * W)
        
        # check convergence (criterion in 1.3)
        # optimize
        objold = obj
        obj = norm(X - V * W)^2
        if verbose
            println("iter = $iter, obj = $obj")
        end
        if abs(obj - objold) ≤ tolfun * (abs(objold) + 1) 
            niter = iter
            break
        end     
    end
    if niter == maxiter
        # @warning "maxiter reached without convergence"
        @warn "SCREAM!!!! maxiter reached without convergence"
    end 
    # Output
    V, W, obj, niter
end

nnmf_prototype (generic function with 1 method)

In [7]:
# load X and starting point
X = readdlm("nnmf-2429-by-361-face.txt")
V0full = readdlm("V0.txt")
W0full = readdlm("W0.txt")

50×361 Matrix{Float64}:
 0.5634    0.32048   0.067202   0.25107   …  0.4476     0.12656   0.13688
 0.71473   0.25423   0.084735   0.84327      0.20947    0.32729   0.43787
 0.046351  0.041688  0.79152    0.8962       0.6786     0.17112   0.83691
 0.76991   0.13366   0.0061026  0.70588      0.4457     0.012722  0.64244
 0.56335   0.35408   0.62697    0.94923      0.073816   0.62934   0.58596
 0.13659   0.82449   0.17183    0.051802  …  0.87725    0.86247   0.45418
 0.2326    0.92296   0.73855    0.35246      0.015108   0.68424   0.15457
 0.95628   0.97221   0.94922    0.59106      0.49038    0.52232   0.4399
 0.82876   0.33257   0.95808    0.11028      0.66812    0.69979   0.57679
 0.33585   0.59454   0.75948    0.26544      0.34689    0.071471  0.020196
 0.87571   0.83349   0.83207    0.57283   …  0.34609    0.22545   0.2295
 0.27631   0.10171   0.4809     0.10614      0.0039955  0.92647   0.27393
 0.914     0.45783   0.32193    0.65607      0.1204     0.29447   0.88988
 ⋮             

In [10]:
r = 10
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
nnmf_prototype(X, r, V = V0, W = W0, verbose = true)

iter = 0, obj = 4.898772038789471e6
iter = 1, obj = 26397.21637479479
iter = 2, obj = 25984.0874695548
iter = 3, obj = 25869.85809608935
iter = 4, obj = 25760.44327059261
iter = 5, obj = 25648.863265758395
iter = 6, obj = 25527.99041197396
iter = 7, obj = 25389.470351972137
iter = 8, obj = 25223.080057769024
iter = 9, obj = 25016.28160328746
iter = 10, obj = 24754.251587964573
iter = 11, obj = 24420.876956576914
iter = 12, obj = 24001.324903966375
iter = 13, obj = 23486.480241352092
iter = 14, obj = 22878.368744815874
iter = 15, obj = 22193.861436724594
iter = 16, obj = 21463.307632261403
iter = 17, obj = 20723.431260629197
iter = 18, obj = 20008.184575799358
iter = 19, obj = 19342.301525748167
iter = 20, obj = 18739.26730548495
iter = 21, obj = 18202.58327336699
iter = 22, obj = 17728.61456024205
iter = 23, obj = 17309.73262737562
iter = 24, obj = 16936.953319932112
iter = 25, obj = 16601.7120945438
iter = 26, obj = 16296.773096661153
iter = 27, obj = 16016.48238388981
iter = 28, obj 

([0.11851552506050547 0.029940503455096936 … 0.0028830808553836898 0.064209910487126; 0.13176807592569614 0.0004309168162359835 … 0.01486343136309386 0.035028745583671625; … ; 0.052705837558246364 0.04794098005164004 … 0.1700196278798265 0.04626182097825301; 0.12933014679416066 0.01956781040212816 … 0.09435983264067124 0.029580777291877237], [3.385885235846894e-27 1.1745375738681033e-27 … 0.23001933337975108 2.2801686250908547e-6; 0.01089979057338704 1.7007366080212261e-6 … 9.165787020781076e-15 5.134415347265328e-11; … ; 4.0722852963975666e-10 0.00011974641917548936 … 6.611383816440941e-6 1.3798984047397109e-12; 5.698509363463677e-10 6.980500700786396e-9 … 2.2318529440909026e-14 1.8764490698063125e-18], 11730.38800985483, 239)

In [54]:
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
@time nnmf_prototype(X, r, V = V0, W = W0)

  1.858176 seconds (3.35 k allocations: 3.236 GiB, 21.16% gc time)


([0.11851552506050547 0.029940503455096936 … 0.0028830808553836898 0.064209910487126; 0.13176807592569614 0.0004309168162359835 … 0.01486343136309386 0.035028745583671625; … ; 0.052705837558246364 0.04794098005164004 … 0.1700196278798265 0.04626182097825301; 0.12933014679416066 0.01956781040212816 … 0.09435983264067124 0.029580777291877237], [3.385885235846894e-27 1.1745375738681033e-27 … 0.23001933337975108 2.2801686250908547e-6; 0.01089979057338704 1.7007366080212261e-6 … 9.165787020781076e-15 5.134415347265328e-11; … ; 4.0722852963975666e-10 0.00011974641917548936 … 6.611383816440941e-6 1.3798984047397109e-12; 5.698509363463677e-10 6.980500700786396e-9 … 2.2318529440909026e-14 1.8764490698063125e-18], 11730.38800985483, 239)

**Step 2 Optimization**
benchmarking, profiling, ...

In [43]:
function nnmf(
    X       :: Matrix{T}, #T: (specifies type of element in the matrix) abstract float
    r       :: Integer; # any type of integer
    maxiter :: Integer = 1000, 
    tolfun  :: Number = 1e-4,
    V       :: Matrix{T} = rand(T, size(X, 1), r), 
    W       :: Matrix{T} = rand(T, r, size(X, 2)),
    verbose :: Bool = false
    ) where T <: AbstractFloat
    # implementation
    n, m = size(X)

    if verbose
        println("iter = 0, obj = $obj")
    end
    
    niter = maxiter
    storageR = Matrix{T}(undef, r, r)
    storageV1 = similar(V) #num
    storageV2 = similar(V) #denom
    storageW1 = similar(W)
    storageW2 = similar(W)
    storageX = similar(X)
    
    #obj = norm(X - V * W)^2
    mul!(storageX, V, W)
    storageX .= X .- storageX
    obj = norm(storageX)^2
    
    for iter in 1:maxiter

        # V .= V .* (X * W') ./ (V * (W * W'))
        mul!(storageR, W, transpose(W)) #O(nr^2)
        mul!(storageV2, V, storageR)
        mul!(storageV1, X, transpose(W))
        V .= V .* storageV1 ./ storageV2
        
        # W .= W .* (V' * X) ./ ((V' * V) * W)
        mul!(storageR, transpose(V), V)
        mul!(storageW2, storageR, W)
        mul!(storageW1, transpose(V), X)
        W .= W .* storageW1 ./ storageW2
        
        # check convergence (criterion in 1.3)
        objold = obj
        mul!(storageX, V, W) # O(mnr) BLAS 3, faster
        storageX .= X .- storageX # O(mn) BLAS 1, slower
        # combine these two steps using gemm
        # BLAS.gemm!(storageX, V, W, *, *)
        obj = norm(storageX)^2
        
        if verbose
            println("iter = $iter, obj = $obj")
        end
        if abs(obj - objold) ≤ tolfun * (abs(objold) + 1) 
            niter = iter
            break
        end     
    end
    if niter == maxiter
        # @warning "maxiter reached without convergence"
        @warn "SCREAM!!!! maxiter reached without convergence"
    end 
    # Output
    V, W, obj, niter
end

nnmf (generic function with 1 method)

In [47]:
r = 10
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
nnmf(X, r, V = V0, W = W0)

([0.11851552506050547 0.029940503455096936 … 0.0028830808553836898 0.064209910487126; 0.13176807592569614 0.0004309168162359835 … 0.01486343136309386 0.035028745583671625; … ; 0.052705837558246364 0.04794098005164004 … 0.1700196278798265 0.04626182097825301; 0.12933014679416066 0.01956781040212816 … 0.09435983264067124 0.029580777291877237], [3.385885235846894e-27 1.1745375738681033e-27 … 0.23001933337975108 2.2801686250908547e-6; 0.01089979057338704 1.7007366080212261e-6 … 9.165787020781076e-15 5.134415347265328e-11; … ; 4.0722852963975666e-10 0.00011974641917548936 … 6.611383816440941e-6 1.3798984047397109e-12; 5.698509363463677e-10 6.980500700786396e-9 … 2.2318529440909026e-14 1.8764490698063125e-18], 11730.38800985483, 239)

In [49]:
r = 10
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
@time nnmf(X, r, V = V0, W = W0)

  0.745996 seconds (14 allocations: 7.117 MiB)


([0.11851552506050547 0.029940503455096936 … 0.0028830808553836898 0.064209910487126; 0.13176807592569614 0.0004309168162359835 … 0.01486343136309386 0.035028745583671625; … ; 0.052705837558246364 0.04794098005164004 … 0.1700196278798265 0.04626182097825301; 0.12933014679416066 0.01956781040212816 … 0.09435983264067124 0.029580777291877237], [3.385885235846894e-27 1.1745375738681033e-27 … 0.23001933337975108 2.2801686250908547e-6; 0.01089979057338704 1.7007366080212261e-6 … 9.165787020781076e-15 5.134415347265328e-11; … ; 4.0722852963975666e-10 0.00011974641917548936 … 6.611383816440941e-6 1.3798984047397109e-12; 5.698509363463677e-10 6.980500700786396e-9 … 2.2318529440909026e-14 1.8764490698063125e-18], 11730.38800985483, 239)

In [50]:
using Profile
Profile.clear()
@profile for i in 1:10 
    V0 = V0full[:, 1:r]
    W0 = W0full[1:r, :]
    nnmf(X, r, V = V0, W = W0)
end
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
     8         0 In[43]                    27 nnmf(X::Matrix{Float64}, r::Int...
     1         0 In[43]                    33 nnmf(X::Matrix{Float64}, r::Int...
     1         0 In[43]                    34 nnmf(X::Matrix{Float64}, r::Int...
     5         0 In[43]                    35 nnmf(X::Matrix{Float64}, r::Int...
    50         0 In[43]                    36 nnmf(X::Matrix{Float64}, r::Int...
     1         0 In[43]                    39 nnmf(X::Matrix{Float64}, r::Int...
     1         0 In[43]                    40 nnmf(X::Matrix{Float64}, r::Int...
    14         0 In[43]                    42 nnmf(X::Matrix{Float64}, r::Int...
     1         0 In[43]                    46 nnmf(X::Matrix{Float64}, r::Int...
  1231         0 In[43]                    47 nnmf(X::Matrix{Float64}, r::Int...
     2         0 In[43]                    50 nnmf(X::Matrix{Float64}, r::Int...

In [55]:
function nnmf_new(
    X       :: Matrix{T}, #T: (specifies type of element in the matrix) abstract float
    r       :: Integer; # any type of integer
    maxiter :: Integer = 1000, 
    tolfun  :: Number = 1e-4,
    V       :: Matrix{T} = rand(T, size(X, 1), r), 
    W       :: Matrix{T} = rand(T, r, size(X, 2)),
    verbose :: Bool = false
    ) where T <: AbstractFloat
    # implementation
    n, m = size(X)
    
    niter = maxiter
    storageR = Matrix{T}(undef, r, r)
    storageV1 = similar(V) #num
    storageV2 = similar(V) #denom
    storageW1 = similar(W)
    storageW2 = similar(W)
    #storageX = similar(X)
    x2norm = norm(X)^2
    obj = x2norm 
    mul!(storageR, transpose(V), V)
    mul!(storageW2, storageR, W)
    mul!(storageW1, transpose(V), X)
    @inbounds for idx in eachindex(W)
        obj = obj + (storageW2[idx] - 2storageW1[idx]) * W[idx]
    end
    
    if verbose
        println("iter = 0, obj = $obj")
    end

    for iter in 1:maxiter
        # optimize 
        # V .= V .* (X * W') ./ (V * (W * W'))
        mul!(storageR, W, transpose(W)) #O(nr^2)
        mul!(storageV2, V, storageR)
        mul!(storageV1, X, transpose(W))
        V .= V .* storageV1 ./ storageV2
        
        # optimize
        # W .= W .* (V' * X) ./ ((V' * V) * W)
        mul!(storageR, transpose(V), V)
        mul!(storageW2, storageR, W)
        mul!(storageW1, transpose(V), X)
        # W .= W .* storageW1 ./ storageW2
        
        objold = obj
        obj = x2norm 
        @inbounds for idx in eachindex(W)
            obj = obj + (storageW2[idx] - 2storageW1[idx]) * W[idx]
            W[idx] = W[idx] * storageW1[idx] / storageW2[idx]
        end 
    
        if verbose
            println("iter = $iter, obj = $obj")
        end
        if abs(obj - objold) ≤ tolfun * (abs(objold) + 1) 
            niter = iter
            break
        end     
    end
    if niter == maxiter
        # @warning "maxiter reached without convergence"
        @warn "SCREAM!!!! maxiter reached without convergence"
    end 
    # Output
    V, W, obj, niter
end

nnmf_new (generic function with 1 method)

In [56]:
r = 10
V0 = V0full[:, 1:r]
W0 = W0full[1:r, :]
@time nnmf_new(X, r, V = V0, W = W0)

  0.968951 seconds (183.72 k allocations: 9.741 MiB, 50.45% compilation time)


([0.11851552506050547 0.029940503455096936 … 0.0028830808553836898 0.064209910487126; 0.13176807592569614 0.0004309168162359835 … 0.01486343136309386 0.035028745583671625; … ; 0.052705837558246364 0.04794098005164004 … 0.1700196278798265 0.04626182097825301; 0.12933014679416066 0.01956781040212816 … 0.09435983264067124 0.029580777291877237], [3.385885235846894e-27 1.1745375738681033e-27 … 0.23001933337975108 2.2801686250908547e-6; 0.01089979057338704 1.7007366080212261e-6 … 9.165787020781076e-15 5.134415347265328e-11; … ; 4.0722852963975666e-10 0.00011974641917548936 … 6.611383816440941e-6 1.3798984047397109e-12; 5.698509363463677e-10 6.980500700786396e-9 … 2.2318529440909026e-14 1.8764490698063125e-18], 11730.86690575003, 239)

### Q1.4 Non-uniqueness

Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

No

### Q1.5 Fixed point

For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

### Q1.6 Interpret NNMF result

Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

### Q1.7 GPU (optional)

Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in Q1.3.

## Q2. Estimating Kinship Matrix

Consider the numerical task of estimating an $n \times n$ kinship matrix $\Phi$ from an $n \times m$ genotype matrix $\mathbf{G}$. Here $n$ is the number of individuals and $m$ is the number of genetic markers. [Lange et al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6763373/) derived a method of moment estimator of form
$$
    \widehat \Phi_{ij} = \frac{e_{ij} - \sum_{k=1}^m [p_k^2 + (1 - p_k)^2]}{m - \sum_{k=1}^m [p_k^2 + (1 - p_k)^2]}, \quad 1 \le i, j \le n,
$$
where
$$
\begin{eqnarray*}
    e_{ij} &=& \frac{1}{4} \sum_{k=1}^m [g_{ik} g_{jk} + (2 - g_{ik})(2 - g_{jk})] \\
    p_k &=& \frac {1}{2n} \sum_{i=1}^n g_{ik}.
\end{eqnarray*}
$$

### Q2.1 Develop code

Write a function that takes a matrix `G` as input and outputs the method of moment estimator. 
Make your function as efficient (both speed and memory) as possible.    

In [ ]:
function kinship(G::Matrix{T}) where T <: AbstractFloat
    n, m = size(G)
    # TODO: your code here
    Φ = zeros(n, m)
    # output
    Φ
end

In [92]:
n = 1000
m = 10000
G = rand(0.0:2.0, 1000, 10000)

1000×10000 Matrix{Float64}:
 0.0  0.0  2.0  2.0  1.0  2.0  2.0  0.0  …  0.0  2.0  2.0  2.0  2.0  2.0  0.0
 1.0  2.0  0.0  0.0  0.0  1.0  2.0  2.0     2.0  1.0  0.0  1.0  0.0  0.0  2.0
 1.0  1.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  1.0  2.0  2.0  1.0  0.0  1.0
 0.0  1.0  2.0  2.0  2.0  0.0  0.0  1.0     2.0  2.0  0.0  0.0  0.0  2.0  2.0
 1.0  0.0  0.0  1.0  1.0  1.0  2.0  2.0     0.0  0.0  2.0  1.0  1.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  …  0.0  2.0  1.0  0.0  1.0  0.0  0.0
 2.0  2.0  2.0  0.0  2.0  1.0  0.0  0.0     0.0  1.0  1.0  2.0  0.0  2.0  1.0
 1.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0     1.0  1.0  0.0  1.0  0.0  1.0  0.0
 2.0  1.0  2.0  2.0  0.0  2.0  1.0  0.0     1.0  1.0  1.0  1.0  2.0  0.0  0.0
 0.0  0.0  0.0  2.0  2.0  1.0  1.0  0.0     1.0  0.0  2.0  2.0  0.0  1.0  1.0
 0.0  2.0  1.0  2.0  2.0  1.0  0.0  1.0  …  2.0  0.0  0.0  0.0  1.0  2.0  2.0
 0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0     2.0  0.0  1.0  0.0  2.0  2.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  2.0  

In [114]:
stride(G, 2)

1000

In [102]:
p = zeros(1, m)
@benchmark mul!(p, ones(1, n), G)

BenchmarkTools.Trial: 1085 samples with 1 evaluation.
 Range (min … max):  4.147 ms …  14.188 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.494 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.592 ms ± 513.750 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▁▆█▆▅▅▃▂▁                                             
  ▂▂▂▂▂▃▆█████████▇▆▅▄▃▄▃▂▃▄▃▂▂▃▂▂▂▃▂▂▂▃▃▂▂▂▂▂▃▂▂▂▂▂▂▁▂▁▂▂▂▂▂ ▃
  4.15 ms         Histogram: frequency by time        5.81 ms <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [108]:
one = ones(1, m)
num = 0
@benchmark num = dot(p, p) + dot(one - p, one - p) 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   82.080 μs …  24.699 ms  ┊ GC (min … max): 0.00% … 99.47%
 Time  (median):     148.285 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   168.267 μs ± 593.317 μs  ┊ GC (mean ± σ):  8.63% ±  2.43%

                      ▂▅███▇▇▅▄▄▃▃▃▂▂▁▁▂▁▂▁▁ ▁▁                 ▃
  ██▅▁▃▁▁▁▁▆▁▃▁▁▄▃▃▁▁▁████████████████████████████▇▆█▇▇▇▇▇▅▆▆▅▆ █
  82.1 μs       Histogram: log(frequency) by time        246 μs <

 Memory estimate: 156.39 KiB, allocs estimate: 7.

### Q2.2 Correctness

First let's make sure our function yields correct answer. Run your function on a fake genotype matrix

In [ ]:
# generate a fake genotype matrix with entries {0, 1, 2}
Random.seed!(257)
G = rand(0.0:2.0, 1000, 10000)
Φ = kinship(G)

Compare the upper left $5 \times 5$ block to what I got using my implementation

```julia
Φ[1:5, 1:5]
```

```
5×5 Matrix{Float64}:
  0.673584     -0.000762864  -0.00266412   0.00343992   0.00293959
 -0.000762864   0.665178     -0.0101691   -0.0110697    0.00223912
 -0.00266412   -0.0101691     0.665078     0.0102444    0.00253932
  0.00343992   -0.0110697     0.0102444    0.66768     -0.0083679
  0.00293959    0.00223912    0.00253932  -0.0083679    0.663777
```

### Q2.3 Efficiency

In a typical genetic data set, $n$ is at order of $10^3 \sim 10^6$ and $m$ is at order of $10^6 \sim 10^7$. Benchmark your function using the smaller data set $G$ generated in Q2.2. Efficiency (both speed and memory) will be the most important criterion when grading this question.

In [ ]:
# benchmark
@btime kinship($G)

**Hint**: I got `@btime` output
```
82.144 ms (3 allocations: 7.64 MiB)
```